In [ ]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from helpers.dataset import LocalRetriever
from fair_forge.metrics import Bias
from pydantic import SecretStr
import os

In [3]:
from getpass import getpass
guardian_api_key = SecretStr(getpass("Please enter your Judge API key: "))

In [4]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
GUARDIAN_URL = os.environ.get("GUARDIAN_URL","https://runtime.apps.hostmydemo.online")
GUARDIAN_MODEL_NAME = os.environ.get("GUARDIAN_MODEL_NAME","ibm-granite/granite-guardian-3.1-2b")
GUARDIAN_API_KEY = guardian_api_key
dataset = os.environ.get("dataset", "asb")
bias_index = f"{dataset}-bias"
guardian_temperature = 0.01
max_tokens = 5

In [5]:
metrics= Bias.run(
    CustomRetriever,
    guardian_url=GUARDIAN_URL,
    guardian_api_key= GUARDIAN_API_KEY,
    guardian_model= GUARDIAN_MODEL_NAME,
    guardian_temperature= guardian_temperature,
    max_tokens= max_tokens
)

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
def flatten_guard_metrics(metrics):
    flattened = []
    for metric in metrics:
        for risk in metric.risks:
            flattened.append(
                {
                    "session_id": metric.session_id,
                    "qa_id": metric.qa_id,
                    "assistant_id": metric.assistant_id,
                    'bias_guard_is_risk': risk.is_risk,
                    'bias_guard_type': risk.risk_type,
                    'bias_guard_probability': risk.probability
                }
            )
    return flattened
flattened = flatten_guard_metrics(metrics)

In [ ]:
mapping_bias = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "bias_guard_is_risk": {"type": "boolean"},
      "bias_guard_type": {"type": "text"},
      "bias_guard_probability": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}

In [ ]:
recreate_index(bias_index, mapping_bias)

In [ ]:
docs = []
for flatten in flattened:
    docs.append({
            "_index": bias_index,
            "_source": flatten
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")